In [1]:
import json
import pandas as pd
import random

In [2]:
f = open("/home/jovyan/DL/20231012_233628_pr_sharings.json")
data = json.loads(f.read())
f.close()

In [3]:
def generate_data(data):
    conversation_id = 0
    first_prompts = []
    first_answers = []
    for entry in data["Sources"]:
        for sharedLink in entry["ChatgptSharing"]:
            if sharedLink.get("Conversations"):
                prompt_id = 0
                for conversation in sharedLink["Conversations"]:
                    first_prompts.append({"prompt_id" : prompt_id, "conversation_id" : conversation_id, "Prompt" : conversation["Prompt"]})
                    first_answers.append({"answer" : conversation["Answer"]})
                    prompt_id+=1
                conversation_id+=1
    df = pd.DataFrame(first_prompts).join(pd.DataFrame(first_answers)).reset_index(drop=True)
    return df

In [4]:
df = generate_data(data)
df.head()
sampled_df = pd.DataFrame(df, columns=df.columns)
sampled_df.to_excel('/home/jovyan/DL/all_data.xlsx', index=False) 

In [5]:
from lingua import LanguageDetectorBuilder
from tqdm.auto import tqdm
detector = LanguageDetectorBuilder.from_all_spoken_languages().build()

/home/jovyan/conda-envs/pythonenv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
detected_langs = []

for index, row in tqdm(df.iterrows(),total=df.shape[0]):
    lang = detector.detect_language_of(row["Prompt"])
    if lang:
        detected_langs.append(lang.name)
    else:
        detected_langs.append("UNKNOWN")
    
df["lang"] = detected_langs

100%|██████████| 996/996 [00:07<00:00, 131.94it/s]


In [7]:
df_eng = df[df.lang == "ENGLISH"]

In [8]:
df_eng.shape[0]/df.shape[0]

0.9136546184738956

In [9]:
df_eng.shape[0]/df_eng.shape[0]

1.0

In [10]:
modified_prompt = []
import re

for index, row in df_eng.iterrows():
    modified_text = re.sub(r'```[^```]*```', '', row["Prompt"])
    if modified_text == row["Prompt"]:
        modified_prompt.append(False)
    else:
        modified_prompt.append(True)
    
df_eng["modified"] = modified_prompt

/tmp/ipykernel_13581/2947244450.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_eng["modified"] = modified_prompt


In [11]:
print(df_eng["modified"])

2      False
3      False
4      False
5      False
6      False
       ...  
989    False
990    False
991    False
993     True
994    False
Name: modified, Length: 910, dtype: bool


In [12]:
#prompts = generate_data(data)
sampled_data = df_eng[df_eng.prompt_id == 0].sample(50)
data_list = sampled_data[['prompt_id','conversation_id', 'Prompt']]
#sampled_data = random.sample(data_list, 50)
print(data_list)
sampled_df = pd.DataFrame(data_list, columns=df_eng.columns)
sampled_df.to_excel('/home/jovyan/DL/sampled_data.xlsx', index=False) 

     prompt_id  conversation_id  \
567          0              104   
647          0              148   
714          0              180   
926          0              274   
844          0              247   
717          0              182   
857          0              252   
80           0               14   
578          0              106   
698          0              170   
277          0               71   
93           0               24   
688          0              162   
979          0              281   
879          0              257   
706          0              174   
549          0               97   
597          0              115   
219          0               50   
552          0               98   
697          0              169   
716          0              181   
752          0              203   
810          0              228   
127          0               35   
766          0              213   
619          0              132   
232          0      

In [13]:
data_list
    

,prompt_id,conversation_id,Prompt
567,0,104,How can I represent the following JSON respons...
647,0,148,I have some duplication in my TypeScript code....
714,0,180,- Too much Equality (max is 4)\n - String quo...
926,0,274,Write Python code that takes this array:\n\n[\...
844,0,247,"""I am building a JavaScript application to sim..."
717,0,182,Write a DeckGL layer for pie charts
857,0,252,synovial cell SubClassOf Nothing\nsynovial cel...
80,0,14,out2.txtDocumentI would like to solve for the ...
578,0,106,"what is the difference between u""abc"" and U""ab..."
698,0,170,I need some place on the page to render the co...


In [69]:
training_data = []
#sampled_data.set_index(df_eng.index, inplace=True)
index_list = sampled_data.index
for index, entry in df_eng.iterrows():
    #for item in sampled_data:
        #if (entry["prompt_id"] != item["prompt_id"]) & (entry["conversation_id"] != item["conversation_id"]):
            #training_data.append(entry)
    if index not in index_list:
        training_data.append(entry)
        
print(len(training_data))
training_data_list = pd.DataFrame(training_data, columns=df_eng.columns)
#training_data_list.set_index(df_eng.index, inplace=True)
training_data_list.to_excel('/home/jovyan/DL/training_data_3.xlsx', index=True) 

860


In [48]:
import openai
import os
import dataclasses
from tqdm.auto import tqdm

In [49]:
def run_inference_prompt(prompt):


    openai.api_key = ""

    model =  "gpt-4-1106-preview"
    max_tokens = 1024
    stop_strs = ["\ndef", "\n#", "\nif", "\nclass"]
    temperature = 0.0
    num_comps=1
    
    messages = [
        {"role": "system", "content":  "You are psychologist expert in understanding human intentions. Given this text from a software developer, summarize what is the intention of the software developer. Don't talk about details of the content. Limit your response to no more than two sentences."},
        {"role": "user", "content": prompt},
    ]

    response = openai.ChatCompletion.create(
            model=model,
            #prompt=prompt,
            messages = messages,
            temperature=temperature,
            max_tokens=max_tokens,
            top_p=1,
            frequency_penalty=0.0,
            presence_penalty=0.0,
            stop=stop_strs,
            n=num_comps,
            #prompt_terminology = "reworded"
        )
    return response['choices'][0]['message']['content']

In [50]:
gpt_df = data_list.copy()
return_inference = []

for index, row in tqdm(data_list.iterrows(), total=data_list.shape[0]):
    return_inference.append(run_inference_prompt(row["Prompt"]))
    
gpt_df["summary"] = return_inference
    

100%|██████████| 50/50 [03:46<00:00,  4.53s/it]


In [51]:
gpt_df.head()

,prompt_id,conversation_id,Prompt,summary
567,0,104,How can I represent the following JSON respons...,The software developer intends to create a Pyt...
647,0,148,I have some duplication in my TypeScript code....,The software developer intends to eliminate re...
714,0,180,- Too much Equality (max is 4)\n - String quo...,The software developer intends to humorously r...
926,0,274,Write Python code that takes this array:\n\n[\...,The software developer intends to create a Pyt...
844,0,247,"""I am building a JavaScript application to sim...",The software developer intends to create a str...


In [52]:
#sampled_df = pd.DataFrame(gpt_df, columns=df_eng.columns)
gpt_df.to_excel('/home/jovyan/DL/gpt_data_5.xlsx', index=False) 